# Analisador Sintático Baseado em Autômatos Finitos - Parte 4: Implementação

Nesta parte, vamos explorar em detalhes a implementação do mapeamento sintático e da interface gráfica.

In [1]:
# Importar bibliotecas necessárias
import sys
import os
import inspect
sys.path.append('.')

from syntactic_analyzer import SyntacticAnalyzer
from grammar import classify_word, CATEGORIES, is_linking_verb

## 1. Mapeamento Sintático

O mapeamento sintático é o processo de atribuir funções sintáticas (sujeito, verbo, objeto, etc.) aos tokens com base em sua posição e categoria gramatical. Este processo é implementado no método `_map_syntactic_functions` da classe `SyntacticAnalyzer`.

O método usa uma máquina de estados para acompanhar o contexto sintático durante a análise da frase. Cada estado representa uma posição específica na estrutura da frase:

In [2]:
# Visualizar o código do método de mapeamento sintático
map_syntactic_functions_code = inspect.getsource(SyntacticAnalyzer._map_syntactic_functions)
print(map_syntactic_functions_code)

    def _map_syntactic_functions(self, tokens):
        """
        Mapeia tokens para funções sintáticas com base em sua posição e classe gramatical.
        Esta é uma simplificação, pois a análise sintática real é muito mais complexa.
        """
        mapped_tokens = []
        state = 0  # Estado atual no processo de mapeamento
        
        for token, category in tokens:
            if state == 0:  # Esperando sujeito ou determinante do sujeito
                if category == "ARTIGO":
                    mapped_tokens.append((token, category, "DET_SUJEITO"))
                    state = 1
                elif category == "SUBSTANTIVO":
                    mapped_tokens.append((token, category, "SUJEITO"))
                    state = 2
                elif category == "PRONOME":
                    mapped_tokens.append((token, category, "SUJEITO"))
                    state = 2
                else:
                    mapped_tokens.append((token, category, "DESCONHECIDO"))
  

### 1.1 Estados do Mapeamento Sintático

O método `_map_syntactic_functions` usa os seguintes estados:

- **Estado 0**: Esperando sujeito ou determinante do sujeito
- **Estado 1**: Após reconhecer determinante do sujeito, esperando sujeito
- **Estado 2**: Após reconhecer sujeito, esperando verbo
- **Estado 3**: Após reconhecer verbo, esperando objeto, advérbio, preposição ou pontuação
- **Estado 4**: Após reconhecer determinante do objeto, esperando objeto
- **Estado 5**: Após reconhecer objeto ou advérbio, esperando preposição, advérbio ou pontuação
- **Estado 6**: Após reconhecer preposição, esperando determinante ou objeto indireto
- **Estado 7**: Após reconhecer determinante do objeto indireto, esperando objeto indireto
- **Estado 8**: Após reconhecer objeto indireto, esperando advérbio ou pontuação
- **Estado 9**: Estado final (após pontuação)

Cada estado tem regras específicas para mapear tokens para funções sintáticas e para determinar o próximo estado.

In [3]:
# Demonstrar o mapeamento sintático com um exemplo
analyzer = SyntacticAnalyzer()
frase = "Eu liguei ao diretor ontem."

# Tokenizar e classificar
tokens = analyzer._tokenize_and_classify(frase)
print("Tokens classificados:")
for token, category in tokens:
    print(f"  - '{token}': {category}")

# Mapear funções sintáticas
syntactic_tokens = analyzer._map_syntactic_functions(tokens)
print("\nTokens com funções sintáticas:")
for token, category, function in syntactic_tokens:
    print(f"  - '{token}': {category} → {function}")

Tokens classificados:
  - 'eu': PRONOME
  - 'liguei': VERBO
  - 'ao': PREPOSICAO
  - 'diretor': SUBSTANTIVO
  - 'ontem': ADVERBIO
  - '.': PONTUACAO

Tokens com funções sintáticas:
  - 'eu': PRONOME → SUJEITO
  - 'liguei': VERBO → VERBO
  - 'ao': PREPOSICAO → PREPOSICAO
  - 'diretor': SUBSTANTIVO → OBJ_INDIRETO
  - 'ontem': ADVERBIO → ADVERBIO
  - '.': PONTUACAO → PONTUACAO


## 2. Tokenização e Classificação

Antes do mapeamento sintático, a frase precisa ser tokenizada (dividida em palavras) e cada token precisa ser classificado em uma categoria gramatical. Este processo é implementado no método `_tokenize_and_classify` da classe `SyntacticAnalyzer`.

In [4]:
# Visualizar o código do método de tokenização e classificação
tokenize_and_classify_code = inspect.getsource(SyntacticAnalyzer._tokenize_and_classify)
print(tokenize_and_classify_code)

    def _tokenize_and_classify(self, sentence):
        """
        Tokeniza e classifica as palavras da frase em categorias gramaticais.
        """
        # Tokenização simples por espaços e remoção de pontuação
        tokens = []
        current_token = ""
        
        for char in sentence:
            if char.isalnum() or char in "áàâãéèêíìîóòôõúùûçÁÀÂÃÉÈÊÍÌÎÓÒÔÕÚÙÛÇ":
                current_token += char
            else:
                if current_token:
                    tokens.append(current_token.lower())
                    current_token = ""
                if not char.isspace():  # Se não for espaço, é pontuação
                    tokens.append(char)
        
        if current_token:  # Adicionar o último token se existir
            tokens.append(current_token.lower())
        
        # Classificar cada token
        classified_tokens = []
        i = 0
        while i < len(tokens):
            token = tokens[i]
            # Tratamento especial para contraçõe

### 2.1 Classificação Gramatical

A classificação gramatical é realizada pela função `classify_word` no módulo `grammar.py`. Esta função verifica se a palavra está em alguma das categorias gramaticais definidas no dicionário `CATEGORIES`.

In [5]:
# Demonstrar a classificação gramatical
palavras = ["o", "gato", "come", "peixe", "rapidamente", "ao", "diretor", "ontem", "é", "bonito"]

print("Classificação gramatical:")
for palavra in palavras:
    categoria = classify_word(palavra)
    print(f"  - '{palavra}': {categoria}")

Classificação gramatical:
  - 'o': ARTIGO
  - 'gato': SUBSTANTIVO
  - 'come': VERBO
  - 'peixe': SUBSTANTIVO
  - 'rapidamente': ADVERBIO
  - 'ao': PREPOSICAO
  - 'diretor': SUBSTANTIVO
  - 'ontem': ADVERBIO
  - 'é': VERBO
  - 'bonito': ADJETIVO


## 3. Interface Gráfica

O sistema inclui uma interface gráfica implementada no módulo `automaton_gui.py`. Esta interface permite ao usuário inserir frases, visualizar a análise sintática e ver o autômato com o caminho percorrido.

A interface gráfica é construída usando a biblioteca Tkinter e inclui os seguintes componentes:

- Campo de entrada para a frase
- Botão para iniciar a análise
- Área de texto para mostrar o resultado da análise
- Canvas para mostrar a imagem do autômato
- Canvas para mostrar a animação do caminho percorrido

O método principal que realiza a análise na interface gráfica é `_analyze_sentence` da classe `AutomatonGUI`.

### 3.1 Código da Interface Gráfica

Aqui está um trecho do código que mostra como a análise sintática é exibida na interface gráfica:

In [7]:
# Código da interface gráfica (não executável no notebook)
analyze_sentence_code = """
def _analyze_sentence(self, event=None):
    ""Analisa a frase e atualiza a interface.""
    sentence = self.sentence_entry.get().strip()
    
    if not sentence:
        messagebox.showwarning("Aviso", "Por favor, digite uma frase para analisar.")
        return
    
    try:
        # Analisar a frase e gerar visualizações
        image_path, gif_path, is_valid, syntactic_tokens = self.visualizer.visualize_sentence(sentence)
        
        # Atualizar os caminhos das imagens
        self.current_image_path = image_path
        self.current_gif_path = gif_path
        
        # Atualizar a área de texto com a análise
        self.analysis_text.config(state=tk.NORMAL)
        self.analysis_text.delete(1.0, tk.END)
        
        self.analysis_text.insert(tk.END, f"Frase: {sentence}\n\n")
        self.analysis_text.insert(tk.END, "Análise Sintática:\n")
        
        for token, category, function in syntactic_tokens:
            self.analysis_text.insert(tk.END, f"  - '{token}': {category} → {function}\n")
        
        self.analysis_text.insert(tk.END, f"\nResultado da Análise:\n")
        if is_valid:
            self.analysis_text.insert(tk.END, "  ✓ Estrutura sintática válida!\n")
            
            # Identificar as principais partes da oração
            sujeito = [token for token, _, function in syntactic_tokens 
                      if function in ["SUJEITO", "DET_SUJEITO", "MODIFICADOR_SUJEITO"]]
            predicado = [token for token, _, function in syntactic_tokens 
                        if function not in ["SUJEITO", "DET_SUJEITO", "MODIFICADOR_SUJEITO", "PONTUACAO"]]
            
            self.analysis_text.insert(tk.END, "\nPartes da Oração:\n")
            self.analysis_text.insert(tk.END, f"  - Sujeito: {' '.join(sujeito)}\n")
            self.analysis_text.insert(tk.END, f"  - Predicado: {' '.join(predicado)}\n")
            
            # Identificar componentes específicos
            verbo = [token for token, _, function in syntactic_tokens if function == "VERBO"]
            objeto_direto = [token for token, _, function in syntactic_tokens 
                           if function in ["OBJETO", "DET_OBJETO", "MODIFICADOR_OBJETO"]]
            objeto_indireto = [token for token, _, function in syntactic_tokens 
                             if function in ["OBJ_INDIRETO", "DET_OBJ_IND", "MODIFICADOR_OBJ_IND", "PREPOSICAO"]]
            adverbios = [token for token, _, function in syntactic_tokens if function == "ADVERBIO"]
            
            if verbo:
                self.analysis_text.insert(tk.END, f"  - Verbo: {' '.join(verbo)}\n")
            if objeto_direto:
                self.analysis_text.insert(tk.END, f"  - Objeto Direto: {' '.join(objeto_direto)}\n")
            if objeto_indireto:
                self.analysis_text.insert(tk.END, f"  - Objeto Indireto: {' '.join(objeto_indireto)}\n")
            if adverbios:
                self.analysis_text.insert(tk.END, f"  - Advérbio: {' '.join(adverbios)}\n")
        else:
            self.analysis_text.insert(tk.END, "  ✗ Estrutura sintática inválida!\n")
            self.analysis_text.insert(tk.END, "  A frase não segue um padrão sintático reconhecido.\n")
        
        self.analysis_text.config(state=tk.DISABLED)
        
        # Atualizar as imagens
        self._update_images(image_path, gif_path)
        
    except Exception as e:
        messagebox.showerror("Erro", f"Ocorreu um erro ao analisar a frase: {str(e)}")
"""

print(analyze_sentence_code)


def _analyze_sentence(self, event=None):
    ""Analisa a frase e atualiza a interface.""
    sentence = self.sentence_entry.get().strip()

    if not sentence:
        messagebox.showwarning("Aviso", "Por favor, digite uma frase para analisar.")
        return

    try:
        # Analisar a frase e gerar visualizações
        image_path, gif_path, is_valid, syntactic_tokens = self.visualizer.visualize_sentence(sentence)

        # Atualizar os caminhos das imagens
        self.current_image_path = image_path
        self.current_gif_path = gif_path

        # Atualizar a área de texto com a análise
        self.analysis_text.config(state=tk.NORMAL)
        self.analysis_text.delete(1.0, tk.END)

        self.analysis_text.insert(tk.END, f"Frase: {sentence}

")
        self.analysis_text.insert(tk.END, "Análise Sintática:
")

        for token, category, function in syntactic_tokens:
            self.analysis_text.insert(tk.END, f"  - '{token}': {category} → {function}
")

        self

## 4. Visualização do Autômato

A visualização do autômato é implementada no módulo `visualize_automaton.py`. Este módulo usa a biblioteca Graphviz para criar representações gráficas do autômato e do caminho percorrido durante a análise.

A visualização inclui:

- Uma imagem estática do autômato com o caminho destacado
- Uma animação GIF mostrando o processo de análise passo a passo

O método principal que cria a visualização é `create_automaton_graph` da classe `AutomatonVisualizer`.

### 4.1 Código da Visualização

Aqui está um trecho do código que mostra como o autômato é visualizado:

In [ ]:
# Código da visualização do autômato (não executável no notebook)
create_automaton_graph_code = """
def create_automaton_graph(self, path=None, highlight_path=True):
    """
    Cria um grafo do autômato usando Graphviz.
    
    Args:
        path: Lista de tuplas (estado, símbolo) representando o caminho percorrido
        highlight_path: Se True, destaca o caminho percorrido
        
    Returns:
        Objeto Digraph do Graphviz
    """
    # Criar um novo grafo direcionado
    dot = Digraph(comment='Autômato Sintático')
    
    # Configurar o grafo
    dot.attr(rankdir='LR', size='8,5')
    dot.attr('node', shape='circle')
    
    # Definir os estados
    states = ['q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9']
    
    # Adicionar estados ao grafo
    for state in states:
        # Verificar se o estado está no caminho
        in_path = False
        if path:
            for s, _ in path:
                if s == state:
                    in_path = True
                    break
        
        # Configurar o estilo do estado
        if state == 'q9':  # Estado final
            if in_path and highlight_path:
                dot.node(state, style='filled', color='red', fillcolor='lightpink', shape='doublecircle')
            else:
                dot.node(state, shape='doublecircle')
        else:  # Estados não-finais
            if in_path and highlight_path:
                dot.node(state, style='filled', color='red', fillcolor='lightpink')
            else:
                dot.node(state)
    
    # Definir as transições
    transitions = [
        ('q0', 'q1', 'DET_SUJEITO'),
        ('q0', 'q2', 'SUJEITO'),
        ('q1', 'q2', 'SUJEITO'),
        ('q2', 'q3', 'VERBO'),
        ('q3', 'q4', 'DET_OBJETO'),
        ('q3', 'q5', 'OBJETO'),
        ('q3', 'q5', 'PREDICATIVO'),
        ('q3', 'q5', 'VERBO_INFINITIVO'),
        ('q3', 'q5', 'ADVERBIO'),
        ('q3', 'q6', 'PREPOSICAO'),
        ('q3', 'q9', 'PONTUACAO'),
        ('q4', 'q5', 'OBJETO'),
        ('q5', 'q6', 'PREPOSICAO'),
        ('q5', 'q5', 'ADVERBIO'),
        ('q5', 'q9', 'PONTUACAO'),
        ('q6', 'q7', 'DET_OBJ_IND'),
        ('q6', 'q8', 'OBJ_INDIRETO'),
        ('q7', 'q8', 'OBJ_INDIRETO'),
        ('q8', 'q8', 'ADVERBIO'),
        ('q8', 'q9', 'PONTUACAO')
    ]
    
    # Adicionar transições ao grafo
    for src, dst, label in transitions:
        # Verificar se a transição está no caminho
        in_path = False
        if path:
            for i in range(1, len(path)):
                prev_state, symbol = path[i-1][0], path[i][1]
                curr_state = path[i][0]
                if prev_state == src and curr_state == dst and symbol == label:
                    in_path = True
                    break
        
        # Configurar o estilo da transição
        if in_path and highlight_path:
            dot.edge(src, dst, label=label, color='red', penwidth='2.0')
        else:
            dot.edge(src, dst, label=label)
    
    return dot
"""

print(create_automaton_graph_code)

## 5. Executando a Interface Gráfica

Para executar a interface gráfica, você pode usar o seguinte comando no terminal:

```bash
python automaton_gui.py
```

Isso abrirá uma janela com a interface gráfica, onde você pode inserir frases para análise e ver o resultado, incluindo a visualização do autômato.